In [ ]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

True

In [ ]:
import pandas as pd
df= pd.read_csv('https://raw.githubusercontent.com/pycaret/pycaret/master/datasets/amazon.csv')

In [ ]:
df.head()

,reviewText,Positive
0,This is a one of the best apps acording to a b...,1
1,This is a pretty good version of the game for ...,1
2,this is a really cool game. there are a bunch ...,1
3,"This is a silly game and can be frustrating, b...",1
4,This is a terrific game on any pad. Hrs of fun...,1


In [ ]:
df

,reviewText,Positive
0,This is a one of the best apps acording to a b...,1
1,This is a pretty good version of the game for ...,1
2,this is a really cool game. there are a bunch ...,1
3,"This is a silly game and can be frustrating, b...",1
4,This is a terrific game on any pad. Hrs of fun...,1
...,...,...
19995,this app is fricken stupid.it froze on the kin...,0
19996,Please add me!!!!! I need neighbors! Ginger101...,1
19997,love it! this game. is awesome. wish it had m...,1
19998,I love love love this app on my side of fashio...,1


In [ ]:
df['reviewText'][0]

'This is a one of the best apps acording to a bunch of people and I agree it has bombs eggs pigs TNT king pigs and realustic stuff'

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
analyzer.polarity_scores('This is a one of the best apps acording to a bunch of people and I agree it has bombs eggs pigs TNT king pigs and realustic stuff')

{'neg': 0.0, 'neu': 0.774, 'pos': 0.226, 'compound': 0.7717}

In [ ]:
analyzer.polarity_scores(df['reviewText'][1999])

{'neg': 0.0, 'neu': 0.665, 'pos': 0.335, 'compound': 0.8779}

In [ ]:
analyzer.polarity_scores(df['reviewText'][4])

{'neg': 0.0, 'neu': 0.478, 'pos': 0.522, 'compound': 0.9552}

In [ ]:
df['reviewText'][10]

"This is definitely a great game.  I have to get my 6-year-old grand-nephew to teach me the tricks.  I have figured out some of them, but some configurations are tough to beat.  I don't particularly care spending about 45 minutes  completing a level, but "

In [ ]:
analyzer.polarity_scores(df['reviewText'][2496])

{'neg': 0.187, 'neu': 0.707, 'pos': 0.106, 'compound': -0.5095}

In [ ]:
def get_sentiment(text):
  scores = analyzer.polarity_scores(text)
  if scores ['pos'] > 0:
    sentiment = 1
  else:
    sentiment = 0
  return sentiment


In [ ]:
get_sentiment(df['reviewText'][2496])

1

In [ ]:
get_sentiment(df['reviewText'][24])

1

In [ ]:
df['sentiment'] = df['reviewText'].apply(get_sentiment)
df

,reviewText,Positive,sentiment
0,This is a one of the best apps acording to a b...,1,1
1,This is a pretty good version of the game for ...,1,1
2,this is a really cool game. there are a bunch ...,1,1
3,"This is a silly game and can be frustrating, b...",1,1
4,This is a terrific game on any pad. Hrs of fun...,1,1
...,...,...,...
19995,this app is fricken stupid.it froze on the kin...,0,0
19996,Please add me!!!!! I need neighbors! Ginger101...,1,1
19997,love it! this game. is awesome. wish it had m...,1,1
19998,I love love love this app on my side of fashio...,1,1


In [ ]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(df['Positive'],df['sentiment']))

[[ 1377  3390]
 [  620 14613]]


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(df['Positive'],df['sentiment']))

              precision    recall  f1-score   support

           0       0.69      0.29      0.41      4767
           1       0.81      0.96      0.88     15233

    accuracy                           0.80     20000
   macro avg       0.75      0.62      0.64     20000
weighted avg       0.78      0.80      0.77     20000



In [ ]:
import re
from nltk.corpus import stopwords

def preprocess_text(text):
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub('[^a-zA-Z]', ' ', text)  # Remove numbers and special characters
    text = text.lower()  # Convert to lowercase
    text = ' '.join([word for word in text.split() if word not in stopwords.words('english')])  # Remove stop words
    return text


In [ ]:
df['clean_text'] = df['reviewText'].apply(preprocess_text)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_features=5000)
tfidf_matrix = tfidf_vectorizer.fit_transform(df['clean_text'])

df_tfidf = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())
df = pd.concat([df, df_tfidf], axis=1)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix, df['Positive'], test_size=0.2, random_state=42)

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
predictions = model.predict(X_test)

print("Random Forest Accuracy:", accuracy_score(y_test, predictions))


Random Forest Accuracy: 0.87


In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {'n_estimators': [50, 100, 200], 'max_depth': [None, 10, 20]}
grid_search = GridSearchCV(RandomForestClassifier(), param_grid, cv=5)
grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_
predictions = best_model.predict(X_test)

print("Tuned Random Forest Accuracy:", accuracy_score(y_test, predictions))


Tuned Random Forest Accuracy: 0.8725


In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression


ensemble_model = VotingClassifier(estimators=[('lr', LogisticRegression()), ('rf', RandomForestClassifier())], voting='hard')
ensemble_model.fit(X_train, y_train)
predictions = ensemble_model.predict(X_test)

print("Ensemble Model Accuracy:", accuracy_score(y_test, predictions))


Ensemble Model Accuracy: 0.8935
